In [1]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

In [3]:
import pickle
import sys
sys.path.insert(0, '../../../summarisation/preprocess')
import vectorizer

from random import shuffle

df, vecs = pickle.load(open('../../../summarisation/preprocess/pico_vecdf_mindf10.p', 'rb'))

In [11]:
data = pickle.load(open('data_pico_new.p', 'rb'))

In [21]:
lang_1 = vecs['outcome']

In [22]:
import re

joined = [[lang_1.idx2word[y] for y in sent] for sent in lang_1.unpad_X]
joined = [list(filter(lambda x: '\n' not in x and '[0]' not in x, sent)) for sent in joined]
joined = [' '.join(sent) for sent in joined]
joined = [re.sub(r'\. qqq \.', '.', sent) for sent in joined]
joined = [sent.split(' ') for sent in joined]

In [23]:
from itertools import groupby
new_sents = []
for sent in joined :
    sent = [x for x in sent if x not in ['<SOS>', '<EOS>'] and '\n' not in x]
    sents = [list(group) for k, group in groupby(sent, lambda x: x == '.' or x == '?') if not k]
    new_sents.append(sents)

In [24]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

In [25]:
process_sents = []
import re
stops = stopwords.words('english')
for doc in new_sents :
    changes = []
    for sent in doc :
        sent = [re.sub('[^a-zA-Z0-9]', '', x).strip() for x in sent]
        sent = [x for x in sent if len(x) > 0]
        sent = [word for word in sent if word not in stops]
        sent = [lmtzr.lemmatize(word) for word in sent]
        changes.append(sent)
    process_sents.append(changes)

In [26]:
idx_cohen = []
word2idx = data['word2idx']
for doc in process_sents :
    idxs = []
    for sent in doc :
        nline = []
        for word in sent :
            if any(char.isdigit() for char in word) :
                nline.append(word2idx['qqq'])
            elif word in word2idx :
                nline.append(word2idx[word])
            else :
                nline.append(word2idx['unk'])
        idxs.append(nline)
    idx_cohen.append(idxs)

In [27]:
data['outcome'] = idx_cohen

In [37]:
pickle.dump(data, open('data_pico_total.p', 'wb'))

In [43]:
[data['idx2word'][y] for y in data['intervention'][-1][0]]

['qqq',
 'unk',
 'hydrogel',
 'dressing',
 'derived',
 'unk',
 'plant',
 'applied',
 'daily',
 'n',
 'qqq',
 'qqq',
 'saline',
 'gauze',
 'dressing',
 'applied',
 'daily',
 'n',
 'qqq',
 'duration',
 'qqq',
 'week',
 'unless',
 'wound',
 'healed',
 'earlier']